# 5.1
## Keras スタイルで CNN をプログラミングする (p.270)

In [1]:
# 5-01-9 Fashion-MNIST の読み込みと前処理
# p.270

from tensorflow.keras import datasets

(x_train, t_train), (x_test, t_test) = datasets.fashion_mnist.load_data()

x_train = x_train / 255
x_test = x_test / 255

# Reshape (-1, 28, 28) --> (-1, 28, 28, 1)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

4431872/4422102 [==============================] - 0s 0us/step


In [2]:
# 5-01-10 CNN モデルの定義
# p.271-274

from tensorflow.keras import models, layers, optimizers, regularizers

model = models.Sequential()

# 正則化の係数
weight_decay = 1e-4

model.add(
    layers.Conv2D(
        filters=64,
        kernel_size=(3,3),
        input_shape=(28, 28, 1),
        padding='same',
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'
    ))

model.add(
    layers.Conv2D(
        filters=32,
    kernel_size=(3, 3),
    padding='same',
    kernel_regularizer=regularizers.l2(weight_decay),
    activation='relu'
    ))

model.add(
    layers.MaxPooling2D(
        pool_size=(2, 2)
    ))

model.add(
    layers.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        padding='same',
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'
    ))

model.add(
    layers.MaxPooling2D(
        pool_size=(2, 2)
    ))

model.add(layers.Dropout(0.4))

model.add(layers.Flatten())

model.add(
    layers.Dense(
        128,
        activation='relu'
    ))

model.add(
    layers.Dense(
        10,
        activation='softmax'
    ))

In [3]:
learning_rate = 0.1

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=optimizers.SGD(lr=learning_rate),
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        18464     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 16)        4624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 7, 7, 16)          0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [4]:
# 5-01-11 CNN モデルによる学習の実行
# p.276

epoch = 100
batch_size = 64

history = model.fit(
    x_train,
    t_train,
    batch_size=batch_size,
    epochs=epoch,
    verbose=1,
    validation_split=0.2,
    shuffle=True
)

Epoch 1/100
750/750 [==============================] - 13s 5ms/step - loss: 0.7090 - accuracy: 0.7369 - val_loss: 0.4515 - val_accuracy: 0.8364
Epoch 2/100
750/750 [==============================] - 3s 4ms/step - loss: 0.4397 - accuracy: 0.8399 - val_loss: 0.3732 - val_accuracy: 0.8648
Epoch 3/100
750/750 [==============================] - 3s 4ms/step - loss: 0.3796 - accuracy: 0.8633 - val_loss: 0.3196 - val_accuracy: 0.8867
Epoch 4/100
750/750 [==============================] - 3s 4ms/step - loss: 0.3405 - accuracy: 0.8763 - val_loss: 0.2907 - val_accuracy: 0.8975
Epoch 5/100
750/750 [==============================] - 3s 4ms/step - loss: 0.3194 - accuracy: 0.8846 - val_loss: 0.2910 - val_accuracy: 0.8951
Epoch 6/100
750/750 [==============================] - 3s 4ms/step - loss: 0.3020 - accuracy: 0.8918 - val_loss: 0.2706 - val_accuracy: 0.9005
Epoch 7/100
750/750 [==============================] - 3s 4ms/step - loss: 0.2889 - accuracy: 0.8959 - val_loss: 0.2713 - val_accuracy: 0.903

In [5]:
# 5-01-12 テストデータによる学習済みモデルの評価
# p.277

test_loss, test_acc = model.evaluate(x_test, t_test, verbose=0)
print(f'test_loss: {test_loss:.4f}  test_acc: {test_acc:.4f}')

test_loss: 0.2584  test_acc: 0.9256


# [自習] Keras の Functional API で記述する

In [6]:
# 5-01-10 CNN モデルの定義
# p.271-274

import tensorflow as tf
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense

input = tf.keras.Input(shape=(28,28,1), dtype='float32')

# 正則化の係数
weight_decay = 1e-4

H = Conv2D(
        filters=64,
        kernel_size=(3,3),
        padding='same',
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'
        )(input)

H = Conv2D(
        filters=32,
        kernel_size=(3, 3),
        padding='same',
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'
        )(H)

H = MaxPooling2D(pool_size=(2, 2))(H)

H = Conv2D(
        filters=16,
        kernel_size=(3, 3),
        padding='same',
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'
        )(H)

H = MaxPooling2D(pool_size=(2, 2))(H)

H = Dropout(0.4)(H)

H = Flatten()(H)

H = Dense(128, activation='relu')(H)

H = Dense(10,activation='softmax')(H)

model2 = tf.keras.Model(inputs=input, outputs=H)

In [7]:
learning_rate = 0.1

model2.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=optimizers.SGD(lr=learning_rate),
    metrics=['accuracy']
)

model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 64)        640       
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 16)        4624      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                         

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [8]:
epoch = 100
batch_size = 64

history = model2.fit(
    x_train,
    t_train,
    batch_size=batch_size,
    epochs=epoch,
    verbose=1,
    validation_split=0.2,
    shuffle=True
)

Epoch 1/100
750/750 [==============================] - 4s 5ms/step - loss: 0.7273 - accuracy: 0.7302 - val_loss: 0.4422 - val_accuracy: 0.8348
Epoch 2/100
750/750 [==============================] - 3s 4ms/step - loss: 0.4413 - accuracy: 0.8422 - val_loss: 0.3653 - val_accuracy: 0.8673
Epoch 3/100
750/750 [==============================] - 3s 4ms/step - loss: 0.3862 - accuracy: 0.8622 - val_loss: 0.3294 - val_accuracy: 0.8812
Epoch 4/100
750/750 [==============================] - 3s 4ms/step - loss: 0.3557 - accuracy: 0.8713 - val_loss: 0.3010 - val_accuracy: 0.8921
Epoch 5/100
750/750 [==============================] - 3s 4ms/step - loss: 0.3322 - accuracy: 0.8818 - val_loss: 0.2856 - val_accuracy: 0.8982
Epoch 6/100
750/750 [==============================] - 3s 4ms/step - loss: 0.3177 - accuracy: 0.8863 - val_loss: 0.2802 - val_accuracy: 0.8983
Epoch 7/100
750/750 [==============================] - 3s 4ms/step - loss: 0.3026 - accuracy: 0.8911 - val_loss: 0.3010 - val_accuracy: 0.8896

In [9]:
test_loss, test_acc = model2.evaluate(x_test, t_test, verbose=0)
print(f'test_loss: {test_loss:.4f}  test_acc: {test_acc:.4f}')

test_loss: 0.2563  test_acc: 0.9249
